In [1]:
import hickle as hkl
import numpy as np

from lib.read_data import u_desvio as u_true
from lib.read_data import y_desvio as y_true

ny = y_true.shape[1]
nu = u_true.shape[1]
theta = hkl.load("../export/ARX.hkl")

A = theta[:ny, :].T
B = theta[ny:, :].T


def model(y: np.ndarray, u_ext: np.ndarray):
    if y.shape != (ny,):
        raise ValueError(f"y inválido: shape {y.shape}, esperado {(ny,)}")

    if u_ext.shape != (2 * nu,):
        raise ValueError(f"u_ext inválido: shape {u_ext.shape}, esperado {(2 * nu,)}")

    return A @ y + B @ u_ext  # y(k+1)


In [2]:
from scipy.optimize import LinearConstraint, minimize

from lib.read_data import u_max_desvio as u_max
from lib.read_data import u_min_desvio as u_min

Np = 10  # horizonte de predição
Nc = 3  # horizonte de controle
Q = np.full(ny, 0.1)  # peso no erro
R = np.full(nu, 0.0001)  # peso na variação da entrada
ki = np.full(ny, 0.0001)  # peso do integrador


def J(du_seq, y0, sp, u0, ni):
    """
    du_seq: sequência de variações de u
    y0: saída atual
    sp: setpoint
    u0: última entrada aplicada
    """
    du_seq = du_seq.reshape((Nc, nu))
    y_pred = y0
    cost = 0.0

    # reconstrói u a partir de du
    u = u0
    u_hist = []

    for k in range(Nc):
        u = u + du_seq[k]
        u_hist.append(u)

    for k in range(Nc, Np):
        u_hist.append(u_hist[-1])  # mantém último u

    # predição
    for k in range(Np):
        if k == 0:
            u_prev = u0
        else:
            u_prev = u_hist[k - 1]

        u_now = u_hist[k]

        u_ext = np.concatenate([u_prev, u_now])

        y_pred = model(y_pred, u_ext)
        e = y_pred - sp - ni

        cost += e.T @ np.diag(Q) @ e

    # penalidade em Δu
    for k in range(Nc):
        du = du_seq[k]
        cost += du.T @ np.diag(R) @ du

    return cost


def constraints(u_old):
    I = np.eye(nu)

    tril_mask = np.tril(np.ones((Nc, Nc)))
    S = np.kron(tril_mask, I)

    lb = np.tile(u_min, Nc) - np.tile(u_old, Nc)
    ub = np.tile(u_max, Nc) - np.tile(u_old, Nc)

    # Restrição final:  lb ≤ S Δu ≤ ub
    return LinearConstraint(S, lb, ub)  # type: ignore


def MPC(y0, sp, u_old, ni):
    """
    y0: saída atual
    sp: setpoint
    u_old: última entrada aplicada
    """
    du0 = np.zeros((Nc * nu,))  # chute inicial: apenas zero

    # bounds em Δu que respeitam limites de u
    constraint = constraints(u_old)

    res = minimize(
        J, du0, args=(y0, sp, u_old, ni), method="SLSQP", constraints=constraint
    )

    du = res.x.reshape((Nc, nu))[0]

    return u_old + du


# Simulando malha fechada


In [3]:
ncycles = 100
cycles = np.arange(0, ncycles)

# Prealocando histórico de saída e entrada
y = np.zeros((ncycles, ny))
y[0] = y_true[0]
u = np.zeros((ncycles, nu))
u[0] = u_true[0]


disturb = 50

y_sp = np.zeros((ncycles, ny))  # setpoint y
y_sp[0:disturb, :] = np.array([2.60473585, -8.24649468, 8.04911466])
y_sp[disturb:, :] = np.array([1.4135061, -3.58229952, 7.48711208])

u_sp = np.zeros((ncycles, nu))  # setpoint u
u_sp[0:disturb, :] = np.array([-70, -45, 5, 25])
u_sp[disturb:, :] = np.array([20, -15, 2, 30])

ni = 0
for k in range(1, ncycles):
    ni += (y[k - 1] - y_sp[k]) * ki

    u[k] = MPC(y[k - 1], y_sp[k], u[k - 1], ni)
    # u[k] = u_sp

    # Atualiza o estado
    y[k] = model(y[k - 1], np.concatenate([u[k - 1], u[k]]))


In [4]:
from lib.read_data import u_ref, y_ref

# u_ref = np.zeros_like(u_ref)
# y_ref = np.zeros_like(y_ref)

y = y + np.tile(y_ref, (y.shape[0], 1))
y_sp = y_sp + y_ref

u = u + np.tile(u_ref, (u.shape[0], 1))
u_sp = u_sp + u_ref

print(y_sp.shape)


(100, 3)


In [5]:
# Plotando gráficos
from lib.plot import plot_control

plot_control(cycles, u, y, y_sp, u_sp, model_name="MPC2")
